In [309]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [310]:
# Load final cleaned datasets
inventory = pd.read_csv("inventory_cleaned.csv")
order_items = pd.read_csv("order_items_cleaned.csv")
products = pd.read_csv("products_cleaned.csv")
users = pd.read_csv("users_cleaned.csv")
orders =  pd.read_csv("orders_cleaned.csv")



In [311]:
inventory.head()

,inventory_item_id,product_id,stocked_at
0,67971,13844,2022-07-02 07:09:20+00:00
1,67972,13844,2023-12-20 03:28:00+00:00
2,67973,13844,2023-06-04 02:53:00+00:00
3,72863,13844,2021-10-16 22:58:52+00:00
4,72864,13844,2021-08-07 16:33:00+00:00


In [312]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490705 entries, 0 to 490704
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   inventory_item_id  490705 non-null  int64 
 1   product_id         490705 non-null  int64 
 2   stocked_at         490705 non-null  object
dtypes: int64(2), object(1)
memory usage: 11.2+ MB


In [313]:
order_items.head()

,order_items_id,order_id,inventory_item_id,sold_at
0,40993,28204,110590,2023-03-14 03:47:21+00:00
1,51224,35223,138236,2023-12-05 13:25:30+00:00
2,36717,25278,99072,2023-12-22 20:48:19+00:00
3,131061,90241,353798,2022-06-19 16:57:59+00:00
4,154022,106052,415780,2023-09-19 09:13:08+00:00


In [314]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154669 entries, 0 to 154668
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   order_items_id     154669 non-null  int64 
 1   order_id           154669 non-null  int64 
 2   inventory_item_id  154669 non-null  int64 
 3   sold_at            154669 non-null  object
dtypes: int64(3), object(1)
memory usage: 4.7+ MB


In [315]:
products.head()

,product_id,cost,category,name,brand,retail_price,department,sku
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF


In [316]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29094 entries, 0 to 29093
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    29094 non-null  int64  
 1   cost          29094 non-null  float64
 2   category      29094 non-null  object 
 3   name          29094 non-null  object 
 4   brand         29094 non-null  object 
 5   retail_price  29094 non-null  float64
 6   department    29094 non-null  object 
 7   sku           29094 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 1.8+ MB


In [317]:
users.head()

,user_id,age,gender,country
0,457,65,M,Brasil
1,6578,34,F,Brasil
2,36280,13,M,Brasil
3,60193,64,M,Brasil
4,64231,25,F,Brasil


In [318]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  100000 non-null  int64 
 1   age      100000 non-null  int64 
 2   gender   100000 non-null  object
 3   country  100000 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.1+ MB


In [319]:
orders.head()

,order_id,user_id
0,8,5
1,60,44
2,64,46
3,89,65
4,102,76


In [320]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125226 entries, 0 to 125225
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   order_id  125226 non-null  int64
 1   user_id   125226 non-null  int64
dtypes: int64(2)
memory usage: 1.9 MB


In [321]:
# Find min and max for each
stocked_range = (inventory['stocked_at'].min(), inventory['stocked_at'].max())
sold_range = (order_items['sold_at'].min(), order_items['sold_at'].max())

# Display results
print("Inventory Stocked Time Range:", stocked_range)
print("Order Sold Time Range:", sold_range)

Inventory Stocked Time Range: ('2018-11-08 04:14:18+00:00', '2024-01-21 03:29:15.939041+00:00')
Order Sold Time Range: ('2019-01-06 02:25:41+00:00', '2024-01-21 18:02:23.533893+00:00')


In [322]:
# Define order activity window
start_date = pd.Timestamp("2019-01-06", tz='UTC')
end_date = pd.Timestamp("2024-01-19", tz='UTC')

In [323]:
# Convert date columns to datetime
inventory["stocked_at"] = pd.to_datetime(inventory["stocked_at"], errors="coerce", utc=True)

inventory = inventory[
    (inventory["stocked_at"] >= start_date) & (inventory["stocked_at"] <= end_date)]

In [324]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 486259 entries, 0 to 490704
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   inventory_item_id  486259 non-null  int64              
 1   product_id         486259 non-null  int64              
 2   stocked_at         486259 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(2)
memory usage: 14.8 MB


In [325]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154669 entries, 0 to 154668
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   order_items_id     154669 non-null  int64 
 1   order_id           154669 non-null  int64 
 2   inventory_item_id  154669 non-null  int64 
 3   sold_at            154669 non-null  object
dtypes: int64(3), object(1)
memory usage: 4.7+ MB


In [326]:
customer_data = pd.merge(orders, users, on="user_id", how="left")

In [327]:
# Merge inventory with order_items on inventory_item_id (left join to keep all inventory)
merged = pd.merge(inventory, order_items, on="inventory_item_id", how="left")

# Merge with product metadata on product_id
merged = pd.merge(merged, products, on="product_id", how="left")

merged = pd.merge(merged, customer_data, on="order_id", how="left")

# Compute financial metrics on filtered data
merged["revenue"] = merged["retail_price"]
merged["profit"] = merged["retail_price"] - merged["cost"]
merged["profit_margin"] = merged["profit"] / merged["retail_price"]



In [328]:
# Preview result
merged.head()

,inventory_item_id,product_id,stocked_at,order_items_id,order_id,sold_at,cost,category,name,brand,retail_price,department,sku,user_id,age,gender,country,revenue,profit,profit_margin
0,67971,13844,2022-07-02 07:09:20+00:00,NaN,NaN,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,NaN,NaN,NaN,NaN,6.99,4.22196,0.604
1,67972,13844,2023-12-20 03:28:00+00:00,NaN,NaN,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,NaN,NaN,NaN,NaN,6.99,4.22196,0.604
2,67973,13844,2023-06-04 02:53:00+00:00,25190.0,17428.0,2022-07-24 06:33:20+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,13873.0,38.0,F,United States,6.99,4.22196,0.604
3,72863,13844,2021-10-16 22:58:52+00:00,NaN,NaN,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,NaN,NaN,NaN,NaN,6.99,4.22196,0.604
4,72864,13844,2021-08-07 16:33:00+00:00,27016.0,18686.0,2021-11-22 02:19:52+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,14870.0,36.0,F,China,6.99,4.22196,0.604


In [329]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486259 entries, 0 to 486258
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   inventory_item_id  486259 non-null  int64              
 1   product_id         486259 non-null  int64              
 2   stocked_at         486259 non-null  datetime64[ns, UTC]
 3   order_items_id     154669 non-null  float64            
 4   order_id           154669 non-null  float64            
 5   sold_at            154669 non-null  object             
 6   cost               485834 non-null  float64            
 7   category           485834 non-null  object             
 8   name               485834 non-null  object             
 9   brand              485834 non-null  object             
 10  retail_price       485834 non-null  float64            
 11  department         485834 non-null  object             
 12  sku                485834 non-

In [330]:
# Saves the cleaned/transformed DataFrame to CSV file
merged.to_csv ('looker_dataset.csv', index = False, header=True)

from google.colab import files

files.download('looker_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>